In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm.notebook import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
import os
import gc
import warnings
warnings.filterwarnings('ignore')

from config import CONTEST_PATH, TRAIN_CSV, SUPPLEMENT_PATH

/tmp/ipykernel_12144/1927936323.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


# EXPLORATION

In [3]:
for dirname, _, filenames in os.walk(CONTEST_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/sample_submission.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/test.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv
/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv


In [4]:
main_df = pd.read_csv(TRAIN_CSV)
main_df.head(10)

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN
5,950661,*OC(=O)CCCCCCCCC(=O)OC1COC2C(*)COC12,NaN,0.339093,NaN,NaN,NaN
6,1012066,*Nc1ccc([C@H]2[C@@H]3C[C@H]4C[C@@H](C3)C[C@@H]...,NaN,0.347616,NaN,NaN,NaN
7,1022104,*C(=O)Nc1ccc(Oc2ccc(Oc3ccc(NC(=O)c4ccc5c(c4)C(...,NaN,0.377545,NaN,NaN,NaN
8,1722264,*CC(*)(C)C(=O)OCCCCCCCCCOc1ccc2cc(C(=O)Oc3cccc...,NaN,0.359176,NaN,NaN,NaN
9,2265305,*Nc1ccc(-c2ccc(-c3ccc(N*)cc3)cc2)cc1,NaN,0.341878,NaN,NaN,NaN


In [5]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7973 entries, 0 to 7972
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       7973 non-null   int64  
 1   SMILES   7973 non-null   object 
 2   Tg       511 non-null    float64
 3   FFV      7030 non-null   float64
 4   Tc       737 non-null    float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 436.1+ KB


In [6]:
supp1_df = pd.read_csv("/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv")
supp1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SMILES   874 non-null    object 
 1   TC_mean  874 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [7]:
supp2_df = pd.read_csv("/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv")
supp2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SMILES  7208 non-null   object
dtypes: object(1)
memory usage: 56.4+ KB


In [8]:
supp3_df = pd.read_csv("/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv")
supp3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SMILES  46 non-null     object 
 1   Tg      46 non-null     float64
dtypes: float64(1), object(1)
memory usage: 864.0+ bytes


In [9]:
supp4_df = pd.read_csv("/media/azizd/TOSHIBA-HDD/data/kaggle_data/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv")
supp4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SMILES  862 non-null    object 
 1   FFV     862 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.6+ KB


# PREPROCESSING

In [10]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

target_columns = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

In [11]:
# Load and combine all data sources
def load_and_combine_data():
    # Load main training data
    main_df = pd.read_csv(TRAIN_CSV)
    
    # Load supplementary datasets
    supplement_dfs = []
    supplement_path = SUPPLEMENT_PATH
    
    for i in range(1, 5):
        file_path = os.path.join(supplement_path, f'dataset{i}.csv')
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            
            # Ensure required columns exist
            for col in target_columns:
                if col not in df.columns:
                    df[col] = np.nan
                    
            # Only keep relevant columns
            df = df[['SMILES'] + target_columns]
            supplement_dfs.append(df)
    
    # Combine all data
    combined_df = pd.concat([main_df] + supplement_dfs, ignore_index=True)
    
    # Remove duplicate SMILES (keep first occurrence)
    combined_df = combined_df.drop_duplicates(subset=['SMILES'])
    
    # Clean SMILES strings
    combined_df['SMILES'] = combined_df['SMILES'].str.strip()
    
    return combined_df

In [12]:
#!pip install rdkit-pypi tqdm

In [13]:
# Calculate molecular descriptors (RDKit required)
def compute_molecular_features(smiles):
    try:
        from rdkit import Chem
        from rdkit.Chem import Descriptors
        
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.zeros(10)  # Return zeros for invalid SMILES
            
        features = [
            Descriptors.MolWt(mol),                # Molecular weight
            Descriptors.HeavyAtomCount(mol),        # Number of heavy atoms
            Descriptors.NumHAcceptors(mol),         # Hydrogen bond acceptors
            Descriptors.NumHDonors(mol),            # Hydrogen bond donors
            Descriptors.TPSA(mol),                  # Polar surface area
            Descriptors.MolLogP(mol),               # Octanol-water partition coefficient
            Descriptors.RingCount(mol),             # Number of rings
            Descriptors.NumRotatableBonds(mol),     # Rotatable bonds
            Descriptors.FractionCSP3(mol),          # Fraction of sp3 carbons
            Descriptors.NumAromaticRings(mol)       # Aromatic rings
        ]
        return np.array(features)
    
    except ImportError:
        # Fallback to simple features if RDKit not available
        return np.array([
            len(smiles),                           # SMILES length
            smiles.count('C'),                     # Carbon atoms
            smiles.count('O'),                     # Oxygen atoms
            smiles.count('N'),                     # Nitrogen atoms
            smiles.count('('),                     # Branch openings
            smiles.count(')'),                     # Branch closings
            smiles.count('='),                     # Double bonds
            smiles.count('#'),                     # Triple bonds
            smiles.count('1'),                     # Ring closures
            smiles.count('@')                      # Chirality indicators
        ])

In [14]:
# Main data processing pipeline
combined_df = load_and_combine_data()

combined_df.head()

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416.0,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187.0,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN


In [15]:
# Calculate molecular features
print("Computing molecular features...")
tqdm.pandas()
combined_df['features'] = combined_df['SMILES'].progress_apply(compute_molecular_features)

combined_df.head()

Computing molecular features...


  0%|          | 0/10345 [00:00<?, ?it/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/traitlets/config/applicat

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/traitlets/config/applicat

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/traitlets/config/applicat

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/azizd/miniconda3/envs/kaggle-gpu/lib/python3.10/site-packages/traitlets/config/applicat

AttributeError: _ARRAY_API not found

,id,SMILES,Tg,FFV,Tc,Density,Rg,features
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN,"[232.3229999999999, 17.0, 2.0, 0.0, 26.3, 3.98..."
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,"[598.9190000000003, 45.0, 2.0, 2.0, 24.06, 12...."
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,"[1003.2069999999997, 73.0, 9.0, 0.0, 122.27000..."
3,519416.0,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN,"[542.7260000000002, 42.0, 2.0, 2.0, 24.06, 11...."
4,539187.0,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN,"[965.1539999999994, 70.0, 14.0, 0.0, 182.28, 1..."


In [46]:
# Split data
train_val, test_df = train_test_split(combined_df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val, test_size=0.15, random_state=42)

# Build vocabulary from all SMILES
all_smiles = "".join(combined_df['SMILES'])
vocab = sorted(set(all_smiles))
char_to_idx = {char: idx+1 for idx, char in enumerate(vocab)}
char_to_idx['<unk>'] = len(char_to_idx) + 1
max_length = max(len(s) for s in combined_df['SMILES'])

# Calculate normalization parameters from TRAINING SET
property_means = train_df[target_columns].mean(skipna=True)
property_stds = train_df[target_columns].std(skipna=True)

# Handle zero std
property_stds = property_stds.replace(0, 1.0)

# MODEL

In [47]:
class PolymerDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        smile = row['SMILES']
        
        # Encode SMILES sequence
        encoded = [char_to_idx.get(c, char_to_idx['<unk>']) for c in smile]
        padded = np.zeros(max_length, dtype=np.int64)
        padded[:len(encoded)] = encoded
        
        # Convert target values to float and handle NaNs
        target_values = np.zeros(len(target_columns), dtype=np.float32)
        mask = np.zeros(len(target_columns), dtype=bool)
        
        for i, col in enumerate(target_columns):
            value = row[col]
            # Handle numeric values and convert strings to float
            if pd.isna(value):
                target_values[i] = 0.0
                mask[i] = False
            else:
                try:
                    # Convert to float if it's a string representation
                    target_values[i] = float(value)
                    mask[i] = True
                except (TypeError, ValueError):
                    target_values[i] = 0.0
                    mask[i] = False
        
        # Normalize targets using precomputed stats
        normalized_targets = (target_values - property_means.values) / property_stds.values
        
        # Get molecular features
        features = row['features']
        
        return (
            torch.tensor(padded, dtype=torch.long),
            torch.tensor(features, dtype=torch.float),
            torch.tensor(normalized_targets, dtype=torch.float),
            torch.tensor(mask, dtype=torch.float)
        )

In [48]:
# Create datasets
train_data = PolymerDataset(train_df)
val_data = PolymerDataset(val_df)
test_data = PolymerDataset(test_df)

In [49]:
class PolymerPredictor(nn.Module):
    def __init__(self, vocab_size, feature_dim=10, embed_dim=256, 
                 lstm_dim=512, output_dim=5, dropout=0.3):
        super().__init__()
        
        # SMILES Embedding Layer
        self.embedding = nn.Embedding(len(char_to_idx)+1, embed_dim, padding_idx=0)
        
        # BiLSTM Layers
        self.bilstm = nn.LSTM(
            embed_dim,
            lstm_dim // 2,
            bidirectional=True,
            batch_first=True,
            num_layers=2
        )
        
        # Attention Mechanism
        self.attention = nn.Sequential(
            nn.Linear(lstm_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        # Feature Processing
        self.feature_fc = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        # Combined Processing
        self.combined_fc = nn.Sequential(
            nn.Linear(lstm_dim + 128, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )
            
    def forward(self, seq_input, feature_input):
        # Process SMILES sequence
        embedded = self.embedding(seq_input)
        lstm_out, _ = self.bilstm(embedded)
        
        # Attention mechanism
        attn_weights = torch.softmax(self.attention(lstm_out).squeeze(-1), dim=1)
        attn_weights = attn_weights.unsqueeze(-1)
        context = torch.sum(lstm_out * attn_weights, dim=1)
        
        # Process molecular features
        features = self.feature_fc(feature_input)
        
        # Combine representations
        combined = torch.cat([context, features], dim=1)
        
        return self.combined_fc(combined)


In [50]:
class RobustHuberLoss(nn.Module):
    def __init__(self, weights=None, l2_lambda=1e-5, huber_delta=1.0):
        super().__init__()
        self.weights = weights
        self.l2_lambda = l2_lambda
        self.huber = nn.HuberLoss(reduction='none', delta=huber_delta)
        
    def forward(self, outputs, targets, mask, model):
        # Calculate base Huber loss
        huber_loss = self.huber(outputs, targets)
        
        # Apply property weights if provided
        if self.weights is not None:
            huber_loss = huber_loss * self.weights.unsqueeze(0)
        
        # Apply mask to ignore NaN targets
        masked_loss = mask * huber_loss
        loss = masked_loss.sum() / (mask.sum() + 1e-8)
        
        # Add L2 regularization
        if self.l2_lambda > 0:
            l2_reg = torch.tensor(0., device=outputs.device)
            for param in model.parameters():
                l2_reg += torch.norm(param)
            loss += self.l2_lambda * l2_reg
        
        return loss

In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
vocab_size = len(vocab)
model = PolymerPredictor(vocab_size).to(device)


In [52]:
# Calculate property weights based on data availability
train_counts = train_df[target_columns].count()
property_weights = torch.tensor([
    5.0 / train_counts['Tg'], # higher weight due to Tg being weak ring on previous results
    1.0 / train_counts['FFV'],
    1.0 / train_counts['Tc'],
    1.0 / train_counts['Density'],
    1.0 / train_counts['Rg']
], device=device)
property_weights = property_weights / property_weights.sum()

# Initialize loss function and optimizer
criterion = RobustHuberLoss(weights=property_weights, l2_lambda=1e-5, huber_delta=1.0)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [53]:

# Evaluation function
def evaluate_model(model, data_loader, criterion, denorm=True):
    model.eval()
    all_preds = []
    all_targets = []
    all_masks = []
    total_loss = 0.0
    
    with torch.no_grad():
        for seq, features, targets, masks in data_loader:
            seq, features, targets, masks = (
                seq.to(device), features.to(device), 
                targets.to(device), masks.to(device)
            )
            
            outputs = model(seq, features)
            loss = criterion(outputs, targets, masks, model)
            total_loss += loss.item()
            
            # Store for metrics
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
            all_masks.append(masks.cpu().numpy())
    
    # Concatenate all batches
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
    all_masks = np.vstack(all_masks)
    
    # Denormalize if needed
    if denorm:
        all_preds = (all_preds * property_stds.values) + property_means.values
        all_targets = (all_targets * property_stds.values) + property_means.values
    
    # Calculate metrics per property
    metrics = {}
    for i, prop in enumerate(target_columns):
        prop_mask = all_masks[:, i].astype(bool)
        preds = all_preds[prop_mask, i]
        targs = all_targets[prop_mask, i]
        
        if len(preds) > 0:
            metrics[prop] = {
                'MAE': mean_absolute_error(targs, preds),
                'RMSE': np.sqrt(mean_squared_error(targs, preds)),
                'R²': r2_score(targs, preds),
                'Samples': len(preds)
            }
        else:
            metrics[prop] = {
                'MAE': np.nan,
                'RMSE': np.nan,
                'R²': np.nan,
                'Samples': 0
            }
    
    # Calculate overall metrics
    valid_preds = []
    valid_targets = []
    for i in range(all_preds.shape[0]):
        for j in range(all_preds.shape[1]):
            if all_masks[i, j]:
                valid_preds.append(all_preds[i, j])
                valid_targets.append(all_targets[i, j])
    
    valid_preds = np.array(valid_preds)
    valid_targets = np.array(valid_targets)
    
    if len(valid_preds) > 0:
        overall_metrics = {
            'MAE': mean_absolute_error(valid_targets, valid_preds),
            'RMSE': np.sqrt(mean_squared_error(valid_targets, valid_preds)),
            'R²': r2_score(valid_targets, valid_preds),
            'Samples': len(valid_preds)
        }
    else:
        overall_metrics = {
            'MAE': np.nan,
            'RMSE': np.nan,
            'R²': np.nan,
            'Samples': 0
        }
    
    return {
        'loss': total_loss / len(data_loader),
        'per_property': metrics,
        'overall': overall_metrics
    }


In [54]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=300):
    best_val_loss = float('inf')
    patience = 15
    no_improve = 0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for seq, features, targets, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            seq, features, targets, masks = (
                seq.to(device), features.to(device), 
                targets.to(device), masks.to(device)
            )
            
            optimizer.zero_grad()
            outputs = model(seq, features)
            loss = criterion(outputs, targets, masks, model)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation phase
        val_metrics = evaluate_model(model, val_loader, criterion, denorm=False)
        val_loss = val_metrics['loss']
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Print epoch summary
        print(f"\nEpoch {epoch+1:03d}")
        print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")
        print(f"  Overall Val MAE: {val_metrics['overall']['MAE']:.4f}, RMSE: {val_metrics['overall']['RMSE']:.4f}, R²: {val_metrics['overall']['R²']:.4f}")
        
        # Early stopping and model checkpoint
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            no_improve = 0
            print("  ↳ Model improved! Saving checkpoint...")
        else:
            no_improve += 1
            print(f"  ↳ No improvement for {no_improve}/{patience} epochs")
            if no_improve >= patience:
                print("Early stopping triggered!")
                break
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    return model

# Train the model
print("\nStarting model training...")
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=300)


Starting model training...


Epoch 1:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 001
  Train Loss: 0.0657 | Val Loss: 0.0270
  Overall Val MAE: 0.6940, RMSE: 0.9048, R²: 0.0728
  ↳ Model improved! Saving checkpoint...


Epoch 2:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 002
  Train Loss: 0.0299 | Val Loss: 0.0248
  Overall Val MAE: 0.6489, RMSE: 0.8676, R²: 0.1476
  ↳ Model improved! Saving checkpoint...


...


Epoch 095
  Train Loss: 0.0040 | Val Loss: 0.0115
  Overall Val MAE: 0.2538, RMSE: 0.4185, R²: 0.8017
  ↳ No improvement for 13/15 epochs


Epoch 96:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 096
  Train Loss: 0.0040 | Val Loss: 0.0114
  Overall Val MAE: 0.2534, RMSE: 0.4168, R²: 0.8032
  ↳ No improvement for 14/15 epochs


Epoch 97:   0%|          | 0/59 [00:00<?, ?it/s]


Epoch 097
  Train Loss: 0.0040 | Val Loss: 0.0115
  Overall Val MAE: 0.2537, RMSE: 0.4183, R²: 0.8019
  ↳ No improvement for 15/15 epochs
Early stopping triggered!


In [55]:
# Evaluate on validation set
val_metrics = evaluate_model(model, val_loader, criterion, denorm=True)
print("\nValidation Metrics:")
for prop, metrics in val_metrics['per_property'].items():
    print(f"  {prop} ({metrics['Samples']} samples):")
    print(f"    MAE: {metrics['MAE']:.4f}  RMSE: {metrics['RMSE']:.4f}  R²: {metrics['R²']:.4f}")

print("\nOverall Validation:")
print(f"  MAE: {val_metrics['overall']['MAE']:.4f}")
print(f"  RMSE: {val_metrics['overall']['RMSE']:.4f}")
print(f"  R²: {val_metrics['overall']['R²']:.4f}")

# Evaluate on test set
test_metrics = evaluate_model(model, test_loader, criterion, denorm=True)
print("\nTest Set Metrics:")
for prop, metrics in test_metrics['per_property'].items():
    print(f"  {prop} ({metrics['Samples']} samples):")
    print(f"    MAE: {metrics['MAE']:.4f}  RMSE: {metrics['RMSE']:.4f}  R²: {metrics['R²']:.4f}")

print("\nOverall Test:")
print(f"  MAE: {test_metrics['overall']['MAE']:.4f}")
print(f"  RMSE: {test_metrics['overall']['RMSE']:.4f}")
print(f"  R²: {test_metrics['overall']['R²']:.4f}")


Validation Metrics:
  Tg (66 samples):
    MAE: 61.7070  RMSE: 80.7569  R²: 0.3058
  FFV (961 samples):
    MAE: 0.0066  RMSE: 0.0112  R²: 0.8356
  Tc (94 samples):
    MAE: 0.0278  RMSE: 0.0398  R²: 0.8031
  Density (84 samples):
    MAE: 0.0313  RMSE: 0.0504  R²: 0.8545
  Rg (84 samples):
    MAE: 1.7347  RMSE: 2.6822  R²: 0.6912

Overall Validation:
  MAE: 3.2816
  RMSE: 18.2865
  R²: 0.5427

Test Set Metrics:
  Tg (82 samples):
    MAE: 58.7250  RMSE: 72.6959  R²: 0.4536
  FFV (1098 samples):
    MAE: 0.0062  RMSE: 0.0106  R²: 0.8634
  Tc (116 samples):
    MAE: 0.0262  RMSE: 0.0393  R²: 0.8038
  Density (93 samples):
    MAE: 0.0377  RMSE: 0.0546  R²: 0.7301
  Rg (94 samples):
    MAE: 1.7128  RMSE: 2.4187  R²: 0.7317

Overall Test:
  MAE: 3.3647
  RMSE: 17.1049
  R²: 0.6808
